In [1]:
import xml.etree.ElementTree as ET
from collections import defaultdict 
import codecs
import json
import re
import collections
import pprint
import pymongo
from tabulate import tabulate


# Open Street map for Boston
## Reason for choosing Boston: 
### i have choosed Boston as i am living in boston and familiar with streets and locations.  it is intresting for me to see aggregated information which is harder to find by searching online.

In [2]:
OSM_FILE = r'D:/DATA_ANALYSIS_NANO_PLUS_DEGREE/mongoDB/Boston_Map/boston_massachusetts.osm'

# Sample Data

In [18]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import xml.etree.ElementTree as ET  # Use cElementTree or lxml if too slow

SAMPLE_FILE = "sample.osm"

k = 100 # Parameter: take every k-th top level element

def get_element(OSM_FILE, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(OSM_FILE, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


with open(SAMPLE_FILE, 'wb') as output:
    output.write(b'<?xml version="1.0" encoding="UTF-8"?>\n')
#     output.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    output.write('<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='utf-8'))

    output.write('</osm>')

### First i parsed through boston dataset using ElementTree and count number of unique tags using count_tags function defined as below

In [19]:
def count_tags(filename):
    counts = defaultdict(int)
    for event , row in ET.iterparse(filename):
        if event =='end':
            counts[row.tag] += 1
        row.clear()
    return counts
tags = count_tags(SAMPLE_FILE)
pprint.pprint(tags)

defaultdict(<type 'int'>, {'node': 19339, 'nd': 23305, 'member': 41, 'tag': 9154, 'relation': 13, 'way': 3094, 'osm': 1})


### I have used Key_type and Processing_file functions to look at attribute k which contains the addresses and count number of cases as following categories: 1- lower for tags which only contains lower case, 2- lower_colon for tags whihc have a semi colon between their values and 3- peoblemchars for tgas with problematic characters as defined

In [20]:
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>^;\'"\?%#$@\,\. \t\r\n]')

In [21]:
def key_type(element, keys):
    if element.tag == "tag":
        if re.search(lower , element.attrib['k']):
            keys['lower'] += 1 
        elif re.search(lower_colon , element.attrib['k']):
            keys['lower_colon'] += 1
        elif re.search(problemchars , element.attrib['k']):
            keys['problemchars'] += 1
        else:
            keys['other'] += 1
    return keys

def processing_file(filename):
    keys = {"lower" : 0 , "lower_colon" : 0 , "problemchars" : 0 , "other" : 0}
    for _, element in ET.iterparse(filename): 
        keys = key_type(element , keys)
        element.clear()
    return keys

return_keys = processing_file(SAMPLE_FILE)
pprint.pprint(return_keys)

{'lower': 8043, 'lower_colon': 673, 'other': 438, 'problemchars': 0}


### defining a Unique_uder function which will iterate through the dataset and find all the "userid" and add them to a set, so we will only keeping unique values

In [22]:
def Unique_user(filename):
    users = set()
    for _, element in ET.iterparse(filename):
        try:
            users.add(element.attrib['uid'])
        except KeyError:
            pass
        element.clear()
    return users
users = Unique_user(SAMPLE_FILE)
print len(users)

301


# Problems

## Street address abbreviation
#### we have the problem which street name abbreviation is not consistence, we are using regex matching that take the  last element in the string which usually is the street types

#### then using an expected and mapping to fix the street types

In [23]:
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

expected = ["Avenue", "Boulevard", "Commons", "Court", "Drive","Exit", "Lane", "Parkway", 
                         "Place", "Road", "Square", "Street", "Trail"]

mapping = {'Ave'  : 'Avenue',
           'Ave.' : 'Avenue',
           'Blvd' : 'Boulevard',
           'Dr'   : 'Drive',
           'Ext'  : 'Exit',
           'Ln'   : 'Lane',
           'Pkwy' : 'Parkway',
           'rd'   :  'Road',
           'Rd'   : 'Road',
           'Rd.'  : 'Road',
           'St,'  : 'Street',
           'st'   : 'Street',
           'st,'  : 'Road',
           'St.'  : 'Street',
           'St'   : 'Street',
           'ST'   : 'Street',
           'street' :"Street",
           'Ct'   : "Court",
           'Cir'  : "Circle",
           'Cr'   : "Court",
           'ave'  : 'Avenue',
           'Hwg'  : 'Highway',
           'Hwy'  : 'Highway',
           'pl'   : 'Place',
           'Sq.'  : 'Square',
           'Sq,'  : 'Square',
           'Sq'   : "Square"}

check_street_names function search the input string for the regex. If there is a match and it is not within the "expected" list, add the match as a key and add the string to the set.
is_address function looks at the attribute k if k="addre:street" audit functio will return the list that match previous two functions. 

We make function check_zipcode to find a list of zip code with incorrect 5 digit formats.

In [24]:
def check_street_names(street_types, street_name):
    temp = street_type_re.search(street_name)
    if temp:
        street_type = temp.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)
            
def is_address(elem):
    return (elem.attrib['k'] == "addr:street")

def checK_zipcode(wrong_zipcodes, zipcode_input):
    firstDigits = zipcode_input[0:2]
    
    if firstDigits != 02:
        wrong_zipcodes[firstDigits].add(zipcode_input)
    elif not firstDigits.isdigit():
        wrong_zipcodes[firstDigits].add(zipcode_input)
        
def is_zipcode(element):
    return(element.attrib['k'] == "addr:postcode")

### shape_element function will return the list that match previous two functions, and also for Proccess_zipcode we create a dict which holds zipcodes with wrong value based on check_zip_code function

In [25]:
def shape_element(osm_filename):
    osm_file = open(osm_filename , "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_address(tag):
                    check_street_names(street_types, tag.attrib['v'])
    return street_types

def proccess_zipcode(osmFile):
    osmfile = open(osmFile, "r")
    wrong_zipcode = defaultdict(set)
    for event, element in ET.iterparse(osmFile, events = ("start" , )):
        if element.tag == "node" or element.tag == "way":
            for tag in element.iter("tag"):
                if is_zipcode(tag):
                    checK_zipcode(wrong_zipcode,tag.attrib['v'])
    return wrong_zipcode


In [26]:
streets = shape_element(SAMPLE_FILE)

In [32]:
Zipcodes = proccess_zipcode(OSM_FILE)

### using pretty print the output of the shape_element and process_zipcode.

In [28]:
pprint.pprint(dict(streets))

{'Park': set(['Austin Park', 'Greenough Park']),
 'St': set(['Cambridge St', 'Everett St', 'Stewart St', 'Waverly St']),
 'St.': set(['Albion St.']),
 'Way': set(['Artisan Way'])}


In [33]:
pprint.pprint(dict(Zipcodes))

{'01': set(['01125', '01238', '01240', '01250', '01821', '01854', '01944']),
 '02': set(['02026',
            '02026-5036',
            '02043',
            '02108',
            '02109',
            '02110',
            '02110-1301',
            '02111',
            '02113',
            '02114',
            '02114-3203',
            '02115',
            '02116',
            '02118',
            '02119',
            '02120',
            '02121',
            '02122',
            '02124',
            '02125',
            '02126',
            '02127',
            '02128',
            '02129',
            '02130',
            '02130-4803',
            '02131',
            '02131-3025',
            '02131-4931',
            '02132',
            '02132-1239',
            '02132-3226',
            '02134',
            '02134-1305',
            '02134-1306',
            '02134-1307',
            '02134-1311',
            '02134-1312',
            '02134-1313',
            '02134-1316',
        

### fixing the street abbreviation and zipcodes based on the condition which we have defined
### and using -----Updated To----> to show what we have done

In [34]:
import itertools
def fix_street_name(name, mapping, regex):
    temp = regex.search(name)
    if temp:
        street_type = temp.group()
        if street_type in mapping:
            name = re.sub(regex , mapping[street_type] , name)
    return name
for street_type, roads in streets.iteritems():
    for road in list(roads)[1:3]:
        update_name = fix_street_name(road, mapping, street_type_re)
        print road + " --------- Updated To ---------> " + update_name  

Austin Park --------- Updated To ---------> Austin Park
Cambridge St --------- Updated To ---------> Cambridge Street
Everett St --------- Updated To ---------> Everett Street


### Note That we did not have any Zipcode in Sample file which needed to be change so i have used original data for this section

In [35]:
def Fix_zipcodes(zipcode):
    testNum = re.findall('[a-zA-Z]*' , zipcode)
    if testNum:
        testNum = testNum[0]
    testNum.strip()
    if testNum == "MA":
        changedZip = (re.findall(r'\d+', zipcode))
        if changedZip:
            if changedZip.__len__() == 2:
                return (re.findall(r'\d+', zipcode))[0] + "-" +(re.findall(r'\d+', zipcode))[1]
            else:
                return (re.findall(r'\d+', zipcode))[0]
            
for zipcodes, styles in Zipcodes.iteritems():
    for style in styles:
        better_style = Fix_zipcodes(style)
        if better_style != None:
            print style, "-----Chnaged To----->", better_style

MA 02116 -----Chnaged To-----> 02116
MA 02118 -----Chnaged To-----> 02118
MA 02186 -----Chnaged To-----> 02186


# Using MongoDB

### Following condition has been considred to transform data from XML to JSON
- you should process only 2 types of top level tags: "node" and "way"
- all attributes of "node" and "way" should be turned into regular key/value pairs, except:
    - attributes in the CREATED array should be added under a key "created"
    - attributes for latitude and longitude should be added to a "pos" array,
      for use in geospacial indexing. Make sure the values inside "pos" array are floats
      and not strings. 
- if second level tag "k" value contains problematic characters, it should be ignored
- if second level tag "k" value starts with "addr:", it should be added to a dictionary "address"
- if second level tag "k" value does not start with "addr:", but contains ":", you can process it
  same as any other tag.
  
  
### and i only print out the records which have address 

In [4]:
address_regex = re.compile(r'^addr\:')
street_regex = re.compile(r'^street')

CREATED = [ "version", "changeset", "timestamp", "user", "uid"]

def is_address(elem):
    if elem.attrib['k'][:5] == "addr:":
        return True

def shape_element(element):
    node = {}
    if element.tag == "node" or element.tag == "way":
        address_info = {}
        nd_info = []
        #pprint.pprint(element.attrib)
        node["type"] = element.tag
        node["id"] = element.attrib["id"]
        if "visible" in element.attrib.keys():
            node["visible"] = element.attrib["visible"]
        if "lat" in element.attrib.keys():
            node["pos"] = [float(element.attrib['lat']), float(element.attrib['lon'])]
        node["created"] = {"version": element.attrib['version'],
                            "changeset": element.attrib['changeset'],
                            "timestamp": element.attrib['timestamp'],
                            "uid": element.attrib['uid'],
                            "user": element.attrib['user']}
        for tag in element.iter("tag"):
            #print tag.attrib
            p = problemchars.search(tag.attrib['k'])
            if p:
                #print "PROBLEM:", p.group()
                continue
            elif is_address(tag):
                if ":" in tag.attrib['k'][5:]:
                    #print "Bad Address:", tag.attrib['k'], "--", tag.attrib['v']
                    continue
                else:
                    address_info[tag.attrib['k'][5:]] = tag.attrib['v']
                    #print "Good Address:", tag.attrib['k'], "--", tag.attrib['v']
            else:
                node[tag.attrib['k']] = tag.attrib['v']
                #print "Outside:", tag.attrib['k'], "--", tag.attrib['v']
        if address_info != {}:
            node['address'] = address_info
        for tag2 in element.iter("nd"):
            nd_info.append(tag2.attrib['ref'])
            #print tag2.attrib['ref']
        if nd_info != []:
            node['node_refs'] = nd_info
        return node
    else:
        return None


def process_map(file_in, pretty = False):
    # You do not need to change this file
    file_out = "{0}.json".format(file_in)
    data = []
    with codecs.open(file_out, "w") as fo:
        for _, element in ET.iterparse(file_in):
            el = shape_element(element)
            if el and 'address' in el:
                data.append(el)
                if pretty:
                    fo.write(json.dumps(el, indent=2)+"\n")
                else:
                    fo.write(json.dumps(el) + "\n")
    return data

# Checking Data with MongoDB

In [5]:
import os
print 'The original OSM file is {} MB'.format(os.path.getsize(OSM_FILE)/1.0e6) # convert from bytes to megabytes
print 'The JSON file is {} MB'.format(os.path.getsize(OSM_FILE + ".json")/1.0e6) # convert from bytes to megabytes

The original OSM file is 434.862953 MB
The JSON file is 3.847137 MB


### using MongoClient and inserting the data on local host

In [6]:
from pymongo import MongoClient

db_name = 'openstreetmap'

def insert_map(infile , db):
    map = process_map(infile)
    
    for row in map :
        db.map.insert_one(row)
    
    
# Connect to Mongo DB
client = MongoClient('localhost:27017')
db = client[db_name]
insert_map(OSM_FILE, db)

In [7]:
pprint.pprint(db.map.find_one())

{u'_id': ObjectId('58e3b20aa193831420864e15'),
 u'address': {u'city': u'Boston',
              u'housenumber': u'599',
              u'state': u'MA',
              u'street': u'Old Colony Avenue'},
 u'attribution': u'Office of Geographic and Environmental Information (MassGIS)',
 u'created': {u'changeset': u'45689302',
              u'timestamp': u'2017-01-31T16:41:55Z',
              u'uid': u'290680',
              u'user': u'wheelmap_visitor',
              u'version': u'6'},
 u'id': u'69482495',
 u'line': u'Red',
 u'massgis:geom_id': u'26',
 u'name': u'JFK/UMass',
 u'operator': u'MBTA',
 u'pos': [42.32074, -71.052486],
 u'railway': u'station',
 u'source': u'massgis_import_v0.1_20071013192438;massgis_import_v0.1_20071013193425',
 u'type': u'node',
 u'website': u'http://www.mbta.com/',
 u'wheelchair': u'yes',
 u'wikipedia': u'en:JFK/UMass (MBTA station)'}


### Printing number of Nodes and Ways available in the data

In [8]:
print "Number of Nodes: " , db.map.find({'type' : 'node'}).count()
print "Numbre of Ways: " , db.map.find({'type' : 'way'}).count()

Number of Nodes:  17486
Numbre of Ways:  41356


### number of unique user exist in the data

In [9]:
len(db.map.distinct('created.user'))

562

### Top 10 user based on number of posts

In [10]:
pprint.pprint(list(db.map.aggregate([{'$group' : {'_id' : '$created.user',
                                  'COUNT' :{"$sum" : 1}}},
                                    {"$sort" : {"COUNT" : -1}},
                                    {"$limit" : 10}])))

[{u'COUNT': 16751, u'_id': u'jremillard-massgis'},
 {u'COUNT': 11697, u'_id': u'ryebread'},
 {u'COUNT': 1974, u'_id': u'amillar'},
 {u'COUNT': 1876, u'_id': u'srevilak'},
 {u'COUNT': 1701, u'_id': u'mterry'},
 {u'COUNT': 1512, u'_id': u'Shannon Kelly'},
 {u'COUNT': 1414, u'_id': u'woodpeck_repair'},
 {u'COUNT': 1176, u'_id': u'crschmidt'},
 {u'COUNT': 1064, u'_id': u'Peter Dobratz'},
 {u'COUNT': 1043, u'_id': u'morganwahl'}]


## top 10 amenities in Boston


In [11]:
pprint.pprint(list(db.map.aggregate([{'$match': {'amenity': {'$exists': 1}}}, \
                                {'$group': {'_id': '$amenity', \
                                            'count': {'$sum': 1}}}, \
                                {'$sort': {'count': -1}}, \
                                {'$limit': 10}])))

[{u'_id': u'school', u'count': 2324},
 {u'_id': u'restaurant', u'count': 2121},
 {u'_id': u'library', u'count': 1960},
 {u'_id': u'fire_station', u'count': 721},
 {u'_id': u'cafe', u'count': 714},
 {u'_id': u'place_of_worship', u'count': 539},
 {u'_id': u'fast_food', u'count': 469},
 {u'_id': u'police', u'count': 336},
 {u'_id': u'university', u'count': 336},
 {u'_id': u'hospital', u'count': 315}]


### Top 6 restaurant type in Boston

In [12]:
pprint.pprint(list(db.map.aggregate([{"$match":{"amenity":{"$exists":1},
                                 "amenity":"restaurant",}},      
                      {"$group":{"_id":{"Food":"$cuisine"},
                                 "count":{"$sum":1}}},
                      {"$project":{"_id":0,
                                  "Food":"$_id.Food",
                                  "Count":"$count"}},
                      {"$sort":{"Count":-1}}, 
                      {"$limit":6}])))

[{u'Count': 476, u'Food': None},
 {u'Count': 168, u'Food': u'american'},
 {u'Count': 147, u'Food': u'pizza'},
 {u'Count': 119, u'Food': u'chinese'},
 {u'Count': 112, u'Food': u'italian'},
 {u'Count': 112, u'Food': u'mexican'}]


### Top 5 building types in Boston

In [13]:
pprint.pprint(list(db.map.aggregate([
       {'$match': {'building': { '$exists': 1}}}, 
        {'$group': {'_id': '$building',
                    'count': {'$sum': 1}}}, 
        {'$sort': {'count': -1}},
        {'$limit': 5}])))

[{u'_id': u'yes', u'count': 32186},
 {u'_id': u'apartments', u'count': 3115},
 {u'_id': u'house', u'count': 2338},
 {u'_id': u'commercial', u'count': 1505},
 {u'_id': u'university', u'count': 980}]


### list of Convinence Stores sort by number

In [17]:
pprint.pprint(list(db.map.aggregate([{"$match":{"shop":{"$exists":1},
                        "shop":"convenience"}},
            {"$group":{"_id":{"City":"$city", "Name":"$name"},
                       "count":{"$sum":1}}},
            {"$project": {'_id':0, "City":"$_id.City",
                          "Name":"$_id.Name", "Count":"$count"}},
            {"$sort":{"Count":-1}},
                                     {"$limit" : 5}
            ])))

[{u'Count': 28, u'Name': u'Tedeschi Food Shops'},
 {u'Count': 21},
 {u'Count': 14, u'Name': u'7-Eleven'},
 {u'Count': 14, u'Name': u'Neighborhood Market'},
 {u'Count': 7, u'Name': u'Shop and Go American and Spanish Groceries'}]


## Conclusion
### Ideas to improve data quality of Boston OSM:
### while auditing the dataset i have noticed there are minor error caused by human input, the dataset is fairly well-cleaned. by using a logical approach we can have a clean data set which can be used for further analysis specially for recomendation engine system which based on the number of posts for the given amenity and existing information can recomend to a user which amnity per location is better and more accesible. 

### Ideas to improve the quality of the data: 
#### - user enters this info through web and need to enter information manually, if user could use cellphone and location automatically entered woud improve the quality of the data.
#### - in case of using web page to enter the information manually we could have asked for user permission to track the location through chroom and update info automatically.

